# Feature Engineering test ground
![Reverse Engineer](https://crystal-cdn3.crystalcommerce.com/photos/6337507/en_UJy7SBBy4F.png)

In [ ]:
import numpy as np
import pandas as pd
import math
from master_transmuter import *

In [ ]:
cd ..

In [ ]:
all_cards_df = pd.read_csv('scrape/all_vintage_cards.csv', low_memory=False)

In [ ]:
all_cards_df['toughness'].unique()

In [ ]:
tough_mask = all_cards_df['toughness'].apply(lambda x: '*' in str(x))
power_mask = all_cards_df['power'].apply(lambda x: '*' in str(x))
all_cards_df[['name', 'power', 'toughness']][tough_mask | power_mask]

In [2]:
cd ..

C:\Users\alexg\Galvanize\mystic-speculation


In [3]:
from unit_tests import *

In [ ]:
cd scrape

In [ ]:
scraping_progress(['mythic', 'rare'])

In [5]:
cards_df = pd.read_csv('scrape/all_vintage_cards.csv')
creature = cards_df[cards_df['name']=='Arcanis the Omnipotent'].iloc[0]
spell = cards_df[cards_df['name']=='Momentary Blink'].iloc[0]
weird = cards_df[cards_df['name']=='Tarmogoyf'].iloc[0]
tests = pd.concat([creature, spell, weird], axis=1).T
tests

C:\Users\alexg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,set_name,type_line,mana_cost,rarity,oracle_text,power,toughness,loyalty,cmc,set,color_identity,colors,reprint,layout,legalities
4027,e7a7941b-68bf-48b1-a5ed-2013068b486c,Arcanis the Omnipotent,Commander 2017,Legendary Creature — Wizard,{3}{U}{U}{U},rare,{T}: Draw three cards.\r\n{2}{U}{U}: Return Ar...,3,4,NaN,6,c17,['U'],['U'],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."
4929,b15effc6-ecab-4b0f-ad5c-4f7b5dbfa325,Momentary Blink,Magic Online Promos,Instant,{1}{W},common,"Exile target creature you control, then return...",NaN,NaN,NaN,2,prm,"['U', 'W']",['W'],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."
21,8a34e2df-a9ee-4c61-bc4c-518475a39925,Tarmogoyf,Ultimate Box Topper,Creature — Lhurgoyf,{1}{G},mythic,Tarmogoyf's power is equal to the number of ca...,*,1+*,NaN,2,puma,['G'],['G'],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."


In [ ]:
pt_mask = np.array([False, False, True])
tests['variable_pt'] = True
tests['variable_pt'][~pt_mask] = False
tests

In [ ]:
pt_mask = np.array([True, False, False])
tests['p:t'] = tests[pt_mask]['power'].astype('int')/tests[pt_mask]['toughness'].astype('int')
tests

In [ ]:
def pt_type(row):
    if (type(row['power']) == type('str')) and (type(row['toughness']) == type('str')):
        if '*' in row['power']+row['toughness']:
            return 'variable'
        return 'static'
    return 'none'
def power_to_int(row):
    if row['pt_type']=='static': 
        return int(row['power'])
    else:
        return row['power']
def tough_to_int(row):
    if row['pt_type']=='static':
        return int(row['toughness'])
    else:
        return row['toughness']
df = tests.copy()
# Create pt_type feature, convert static pts to ints
df['pt_type'] = df.apply(pt_type, axis=1)
df['power'] = df.apply(power_to_int, axis=1)
df['toughness'] = df.apply(tough_to_int, axis=1)

mask = df['pt_type']=='static'

df['p:t'] = df[mask]['power']/df[mask]['toughness']
df['p+t'] = df[mask]['power']+df[mask]['toughness']
df['p*t'] = df[mask]['power']*df[mask]['toughness']
df['sqrt_pt'] = math.sqrt(df[mask]['p*t'])
df['avg_pt'] = (df[mask]['power']+df[mask]['toughness'])/2
df['cmc:p+t'] = df[mask]['cmc']/df[mask]['p+t']
df['cmc:p*t'] = df[mask]['cmc']/df[mask]['sqrt_pt']
df

In [ ]:
df['p*t'].apply(np.sqrt)

In [ ]:
math.sqrt((df[mask]['power']*df[mask]['toughness']))

In [ ]:
from unit_tests import *

In [ ]:
test_feature_engineering(cards_df)

In [6]:
from scrape.query import *

In [ ]:
test_cards = cards_df[cards_df['rarity']=='mythic']
test_cards[['recent_date','recent_price']] = test_cards.apply(get_recent_price, axis=1).apply(pd.Series)

In [130]:
cards_df[['recent_date','recent_price']] = cards_df.apply(get_recent_price, axis=1).apply(pd.Series)

ProgrammingError: ('(psycopg2.ProgrammingError) relation "uncommon_price_history_2" does not exist\nLINE 1: ...cardname, ph.setname, ph.timestamp, ph.price from uncommon_p...\n                                                             ^\n', 'occurred at index 53') [SQL: "select ph.cardname, ph.setname, ph.timestamp, ph.price from uncommon_price_history_2 ph,      (select max(timestamp) as lastdate       from uncommon_price_history_2 ph2       where ph2.cardname='Kitchen Finks' and ph2.setname='Ultimate Masters') mostrecent where ph.timestamp = mostrecent.lastdate "] (Background on this error at: http://sqlalche.me/e/f405)

In [116]:
test_card.apply(get_recent_price, axis=1)

('Polyraptor', 'Rivals of Ixalan', '1542070311053', 2.3)


2857    (1542070311053, 2.3)
dtype: object

In [122]:
test_card.apply(get_recent_price, axis=1)[:]

('Polyraptor', 'Rivals of Ixalan', '1542070311053', 2.3)


2857    (1542070311053, 2.3)
dtype: object

In [ ]:
write_recent_prices(cards_df, ['mythic', 'rare'])

In [1]:
ls

 Volume in drive C is OS
 Volume Serial Number is 606C-8C38

 Directory of C:\Users\alexg\Galvanize\mystic-speculation\model

11/15/2018  09:17 PM    <DIR>          .
11/15/2018  09:17 PM    <DIR>          ..
11/14/2018  05:34 PM    <DIR>          .ipynb_checkpoints
11/15/2018  06:29 PM    <DIR>          __pycache__
11/15/2018  06:22 PM             6,066 master_transmuter.py
11/15/2018  09:17 PM            31,720 Reverse_Engineer.ipynb
               2 File(s)         37,786 bytes
               4 Dir(s)  10,818,215,936 bytes free
